<a href="https://colab.research.google.com/github/baraajadallah/Project-1/blob/main/Copy_of_dicom_files_assortion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mout the Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install pydicom gdcm pylibjpeg pylibjpeg-libjpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 56.3 MB/s eta 0:00:00


In [ ]:
import os
import pydicom
import shutil

def categorize_and_rename_dicom_axial_no(dicom_path, output_folder):
    # Create output folder
    os.makedirs(output_folder, exist_ok=True)

    # Dictionary to store the number of slices per series
    series_slices_count = {}

    # Iterate through DICOM files in the folder
    for dicom_file in os.listdir(dicom_path):
        dicom_filepath = os.path.join(dicom_path, dicom_file)

        # Check if it's a DICOM file
        if not dicom_filepath.lower().endswith('.dcm'):
            continue

        # Read DICOM metadata
        dicom_data = pydicom.dcmread(dicom_filepath, force=True)

        # Check if 'ImageOrientationPatient' metadata is present
        if 'ImageOrientationPatient' not in dicom_data:
            print(f"Skipping file {dicom_filepath} - Missing ImageOrientationPatient metadata")
            continue

        # Determine orientation
        orientation = dicom_data.ImageOrientationPatient
        is_axial = round(orientation[0], 0) == 1 and orientation[5] == 0

        # Check WindowCenter and WindowWidth for contrast determination
        window_center = int(dicom_data.WindowCenter)
        window_width = int(dicom_data.WindowWidth)
        is_contrast_slice = window_center >= 0 or window_width <= 800

        # Check if it's a non-contrast axial slice
        if not is_contrast_slice and is_axial:
            # Identify the series
            series_id = dicom_data.SeriesInstanceUID

            # Count the number of slices in the series
            if series_id not in series_slices_count:
                series_slices_count[series_id] = 0
            series_slices_count[series_id] += 1

            # Create series folder if not exists
            series_folder = os.path.join(output_folder, f'Series_{series_id}')
            os.makedirs(series_folder, exist_ok=True)

            # Rename the DICOM file and copy to the series folder
            new_filename = f"{series_id}_{dicom_file}"
            target_filepath = os.path.join(series_folder, new_filename)
            shutil.copy(dicom_filepath, target_filepath)

    # Check and remove series folders with less than 10 slices
    for series_id, num_slices in series_slices_count.items():
        if num_slices < 10:
            series_folder = os.path.join(output_folder, f'Series_{series_id}')
            shutil.rmtree(series_folder)

if __name__ == "__main__":
    dicom_path = '/content/gdrive/MyDrive/wanted_scans_2/8079712/w5177093'
    output_folder = '/content/gdrive/MyDrive/only_axial/scan_8079712'
    categorize_and_rename_dicom_axial_no(dicom_path, output_folder)


Skipping file /content/gdrive/MyDrive/wanted_scans_2/8079712/w5177093/view0130.dcm - Missing ImageOrientationPatient metadata
